In [7]:
import ee
ee.Initialize()
import json
from pathlib import Path
import tifffile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

UCD_PATH = Path('C:/Users/hafne/urban_change_detection')
OSCD_PATH = Path('C:/Users/hafne/urban_change_detection/OSCD_dataset')
DRIVE_FOLDER = 'ucd_exports'
SANITY_CHECK_PATH = Path('C:/Users/hafne/urban_change_detection/sentinel1_sanity_ckeck')

CITIES = ['aguasclaras', 'bercy', 'bordeaux', 'nantes', 'paris', 'rennes', 'saclay_e', 'abudhabi', 'cupertino',
          'pisa', 'beihai', 'hongkong', 'beirut', 'mumbai', 'brasilia', 'montpellier', 'norcia', 'rio', 'saclay_w',
          'valencia', 'dubai', 'lasvegas', 'milano', 'chongqing']

ORBITS = {
    'aguasclaras': [24],
    'bercy': [59, 8, 110],
    'bordeaux': [30, 8, 81],
    'nantes': [30, 81],
    'paris': [59, 8, 110],
    'rennes': [30, 81],
    'saclay_e': [59, 8],
    'abudhabi': [130],
    'cupertino': [35, 115, 42],
    'pisa': [15, 168],
    'beihai': [157],
    'hongkong': [11, 113],
    'beirut': [14, 87],
    'mumbai': [34],
    'brasilia': [24],
    'montpellier': [59, 37],
    'norcia': [117, 44, 22, 95],
    'rio': [155],
    'saclay_w': [59, 8, 110],
    'valencia': [30, 103, 8, 110],
    'dubai': [130, 166],
    'lasvegas': [166, 173],
    'milano': [66, 168],
    'chongqing': [55, 164]
}


# length of sentinel 1 time series for mean
TS_LENGTH = 3

In [2]:
# helper functions

def date2str(date: ee.Date) -> str:
    return str(date.format('Y-M-d').getInfo())

def read_in_dates(dates_file: Path):
    with open(dates_file, 'r') as f:
        lines = f.readlines()
    
    def line2date(line):
        year, month, day = line[8:12], line[12:14], line[14:16]
        date = f'{year}-{month}-{day}'
        return ee.Date(date)
    
    dates = [line2date(line) for line in lines]    
    return dates

def read_in_bbox(bbox_file: Path):
    with open(bbox_file) as f:
        geom = json.load(f)['features'][0]['geometry']
    coords = geom['coordinates'][0]
    ll, ur = coords[0], coords[2]
    bbox = ee.Geometry.Rectangle([ll, ur])
    return bbox


def orbit_mean(roi: ee.Geometry, date_range, orbit, pol) -> ee.Image:
    
    # sup-setting data
    col = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(roi) \
        .filterDate(date_range.start(), date_range.end()) \
        .filterMetadata('instrumentMode', 'equals', 'IW') \
        .filterMetadata('relativeOrbitNumber_start', 'equals', orbit) \
        .select(pol)

    # masking noise
    col = col.map(lambda img: img.updateMask(img.gte(-25)).copyProperties(img))

    mean = col.mean() \
        .unitScale(-25, 5) \
        .clamp(0, 1) \
        .unmask() \
        .float()

    return mean

In [8]:
cities = ['mumbai']
for city in cities:
    print(city)
    
    # loading dates
    dates_file = OSCD_PATH / 'images' / city / 'dates.txt'
    t1, t2 = read_in_dates(dates_file)
    
    # computing date ranges
    date_range_t1 = ee.DateRange(t1.advance(-TS_LENGTH, 'month') , t1)
    if city == 'mumbai':
        date_range_t1 = ee.DateRange(t1.advance(-1, 'year'), t1.advance(5, 'month'))
    date_range_t2 = ee.DateRange(t2, t2.advance(TS_LENGTH, 'month'))
    
    # loading bounding box
    bbox_file = OSCD_PATH / 'images' / city / f'{city}.geojson'
    bbox = read_in_bbox(bbox_file)
    
    # getting orbit numbers
    orbits = ORBITS[city]
    
    # download data for each orbit number and t
    for orbit in orbits:
        
        s1_t1 = orbit_mean(bbox, date_range_t1, orbit, 'VV')
        task_t1 = ee.batch.Export.image.toDrive(
            image=s1_t1,
            region=bbox.getInfo()['coordinates'],
            description='PythonToDriveExport',
            folder=DRIVE_FOLDER,
            fileNamePrefix=f'sentinel1_{city}_{orbit}_t1',
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e10,
            fileFormat='GeoTIFF'
        )
        task_t1.start()
    
        s1_t2 = orbit_mean(bbox, date_range_t2, orbit, 'VV')
        task_t2 = ee.batch.Export.image.toDrive(
            image=s1_t2,
            region=bbox.getInfo()['coordinates'],
            description='PythonToDriveExport',
            folder=DRIVE_FOLDER,
            fileNamePrefix=f'sentinel1_{city}_{orbit}_t2',
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e10,
            fileFormat='GeoTIFF'
        )
        task_t2.start()
    
        

mumbai
